In [ ]:
import json
import cryptocompare
import utilities
import collections
import pandas as pd
import networkx

class Exchange(object):
    def __init__(self, name, exchange):
        self.name = name
        self.rates = {}
        self.update(exchange)

    def update(self, exchange):
        if 'www.cryptocompare.com' not in exchange:
            return

        #TODO: Remove retarded values
        #TODO: Filter out bad exchanges
        for entry in exchange['www.cryptocompare.com'].entries:
            for trade in entry.trades:
                if trade[0] == ['BTC', 'USD']:
                    print (self.name, trade)
                self.rates[tuple(trade[0])] = trade[1]


def get_ranks(exchanges):
    rankings = collections.defaultdict(list)

    for exchange in exchanges:
        for pair in exchange.rates:
            rankings[pair] += [(exchange.rates[pair], exchange.name)]

    for pair in rankings:
        rankings[pair] = sorted(rankings[pair])

    return rankings


def calculate_spreads(rankings):
    # TODO: For every coin calculate the maxiumum spread
    percents = {}
    for pair in rankings:
        best = rankings[pair][-1][0]
        percents[pair] = [(rate[0] / best, rate[1]) for rate in rankings[pair]]

    return percents


def to_df(exchanges):
    rows = [(exchange.name, pair[0], pair[1], exchange.rates[pair])
            for exchange in exchanges
            for pair in exchange.rates]
    return pd.DataFrame(rows, columns=('name', 'dst', 'src', 'rate'))


def add_to_graph(frame):
    # Add all the pairs and their exchange rate to the graph
    # TODO: currently this is undirected because of the cryptocompare data
    # TODO: Once you connect the apis, this can have different buy/sell prices
    graph = networkx.Graph()
    for row in frame:
        dst = f"{row.name_x}_{row.dst}"
        src = f"{row.name_x}_{row.src}"

        other_dst = f"{row.name_y}_{row.dst}"
        other_src = f"{row.name_y}_{row.src}"

        graph.add_edge(src, dst, rate=frame.rate_x)
        graph.add_edge(src, other_src, rate=1.0)
        graph.add_edge(dst, other_dst, rate=1.0)

    return graph

if __name__ == '__main__':
    # Task:
    # I wish find the best two exchanges to put my money such that I can buy coin C1 from exchange E1
    # Transfer Coin C1 to E2, sell Coin C1 for C2, Transfer Coin C2 Back To E1 and repurchase C1
    # Notes: within an exchange am I willing to do intermediate trades?

    # Arbitrage graph?


    with open("spread.json") as fp:
        encoder = utilities.get_encoder(cryptocompare.Exchange, cryptocompare.Entry, cryptocompare.Trade)
        exchanges = json.load(fp, object_hook=encoder.decode)
        processed_exchanges = [Exchange(name, exchange) for name, exchange in exchanges.items()]
        rankings = get_ranks(processed_exchanges)
        percents = calculate_spreads(rankings)

        frame = to_df(processed_exchanges)
        merged = frame.merge(frame, on=('dst', 'src'))
        merged['spread'] = merged.rate_x / merged.rate_y - 1
        top_spread = merged.sort_values(by='spread')
        print(top_spread)
        print(add_to_graph(frame))


